In [71]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [72]:
dataset = pd.read_csv('../Datasets/Thyroid/thyroidDF.csv')

In [73]:
diagnoses = {'-': 'negative',
             'A': 'hyperthyroid', 
             'C': 'hyperthyroid', 
             'B': 'hyperthyroid', 
             'D': 'hyperthyroid',
             'E': 'hypothyroid', 
             'F': 'hypothyroid', 
             'G': 'hypothyroid', 
             'H': 'hypothyroid'}

dataset['target'] = dataset['target'].map(diagnoses) # re-mapping
dataset.dropna(subset=['target'], inplace=True)

In [74]:
dataset.drop(['TSH_measured', 'T3_measured', 'TT4_measured', 'T4U_measured', 'FTI_measured', 'TBG_measured', 'patient_id', 'referral_source', 'TBG'], axis=1, inplace=True)

In [75]:
dataset[dataset['age']>100]

dataset.drop(dataset[dataset['age']>100].index, inplace=True)

In [76]:
dataset['sex'] = np.where((dataset.sex.isnull()) & (dataset.pregnant == 't'), 'F', dataset.sex)

In [77]:
dataset.drop('T3', axis=1, inplace=True)

In [78]:
dataset.dropna(inplace=True)

In [79]:
dataset.replace('f', 0, inplace=True)
dataset.replace('t', 1, inplace=True)
dataset.replace('M', 0, inplace=True)
dataset.replace('F', 1, inplace=True)

In [80]:
diagnoses = {'negative': 0,
             'hypothyroid': 1, 
             'hyperthyroid': 2}

dataset['target'] = dataset['target'].map(diagnoses) # re-mapping

# train and test split --> stratified
X = dataset.drop('target', axis=1).copy()
y = dataset['target'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [81]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [92]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9839743589743589

In [93]:
import pickle
with open('./models/thyroid_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [11]:
import pickle
with open('./models/thyroid_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [105]:
len(automl.show_models())

8

# LIME

In [100]:
import lime
import lime.lime_tabular
import tqdm 

In [95]:
continuous_features = ['age', 'TSH', 'TT4', 'T4U', 'FTI']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['negative', 'hypothyroid', 'hyperthyroid'], categorical_features=categorical_features, discretize_continuous=True)

In [96]:
text_x = X_test.values

In [101]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [102]:
import metrics

In [103]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

100%|██████████| 1560/1560 [2:37:17<00:00,  6.05s/it]  


In [107]:
np.save('./explanations/thyroid1.npy', exp1)
np.save('./explanations/thyroid2.npy', exp2)

In [108]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [109]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [110]:
i, s, sb

((100.0, 0, 1560), (0, 1560, 2433600, 0.0), (118, 1560))

In [111]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [112]:
sim

0.26246664645487466

In [114]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in range(len(text_x)):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(3, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, text_x[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

In [ ]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))